In [1]:
#imp dependencies
import pandas as pd
import requests
import numpy as np
from citipy import citipy

#imp api key
from config import weather_api_key

In [2]:
#creating random lat and long
lats = np.random.uniform(-90, 90, size=2000)
lngs = np.random.uniform(-180, 180, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

#list for coordinates
coordinates = list(lat_lngs)

In [3]:
#creating list of cities with nearest cities
cities = []
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

#add unique values to list
    if city not in cities:
        cities.append(city)

In [4]:
#Setting url def for url source with my apy key
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [5]:
#create city data empty list
city_data = []

#set counters
record_count = 1
set_count = 1

#run through cities
for i, city in enumerate(cities):

    #cities in sets of 50
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    #endpoint url
    city_url = url + "&q=" + city.replace(" ","+")

    try:
        #set city weather def and parse json()
        city_weather = requests.get(city_url).json()
        #selecting data were looking for
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
     
        #city details "append" onto city data list
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_description})

#Skip city if error with except
    except:
        pass

In [6]:
#Adding new data to create new DF
city_data_df = pd.DataFrame(city_data)
#Column ordering
new_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_data_df = city_data_df[new_order]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,San Cristobal,VE,7.7669,-72.2250,69.98,71,91,4.00,overcast clouds
1,Rikitea,PF,-23.1203,-134.9692,75.02,72,86,16.31,overcast clouds
2,Butaritari,KI,3.0707,172.7902,81.88,79,19,15.35,few clouds
3,Castro,BR,-24.7911,-50.0119,80.29,58,64,4.23,broken clouds
4,Sao Filipe,CV,14.8961,-24.4956,74.79,75,48,4.90,scattered clouds
5,Padang,ID,-0.9492,100.3543,76.53,97,100,11.34,overcast clouds
6,Port Alfred,ZA,-33.5906,26.8910,63.48,61,87,20.87,overcast clouds
7,Sovetskiy,RU,61.3614,63.5842,5.13,95,100,5.73,overcast clouds
8,Helena,US,46.5927,-112.0361,18.18,58,7,4.00,snow
9,Torbay,CA,47.6666,-52.7314,31.75,96,90,27.63,snow


In [7]:
#export DF as CSV into weather DB folder
output_data_file = "Weather_Database/WeatherPy_Database.csv"
city_data_df.to_csv(output_data_file, index_label="City_ID")